In [6]:
!pip install -q -U kaggle_environments

In [7]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

In [8]:
%%writefile always_rock.py

#Agent which always returns rock
#0 - rock
#1 - paper
#2 - scissors
def always_rock(observation, configuration):
    return 0

Writing always_rock.py


In [9]:
%%writefile always_paper.py

#Agent which always returns paper
def always_paper(observation, configuration):
    return 1

Writing always_paper.py


In [10]:
%%writefile always_scissors.py

#Agent which always returns scissors
def always_scissors(observation, configuration):
    return 2

Writing always_scissors.py


In [11]:
%%writefile random.py
#Agent which randomly returns different values
def random(observation, configuration):
    import random
    return random.randint(0, 2)

Writing random.py


In [12]:
%%writefile copy_opponent_old.py
def copy_opponent_old(observation, configuration):
    import random
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randint(0, 2)

Writing copy_opponent_old.py


In [13]:
%%writefile statistical_old.py
def statistical_old(observation, configuration):
    import random
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Writing statistical_old.py


In [14]:
%%writefile random_strategy_switch.py
def random_strategy_switch(observation, configuration):
    import random
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return 1  # Сounteracts the last move of the opponent
        elif observation.lastOpponentAction == 1:
            return random.randint(1, 2) # Counteracts or copy the last move of the opponent
        elif observation.lastOpponentAction == 2:
            return random.randint(0, 1) # Gives way or counteracts the lasr move of the opponent
    #initial stepделает более сильный ход после противника
    else:
        return random.randint(0, 2)

Writing random_strategy_switch.py


In [15]:
%%writefile counteracting.py
def counteracting(observation, configuration):
#Agent thay always makes stronger move after the opponent
    import random
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return 1
        elif observation.lastOpponentAction == 1:
            return 2
        elif observation.lastOpponentAction == 2:
            return 0
    #initial step
    else:
        return random.randint(0, 2)    

Writing counteracting.py


In [16]:
%%writefile giveaway_gamer.py
def giveaway_gamer(observation, configuration):
#Agent thay always makes weaker move after the opponent
    import random
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return 2
        elif observation.lastOpponentAction == 1:
            return 0
        elif observation.lastOpponentAction == 2:
            return 1
    #initial step
    else:
        return random.randint(0, 2)

Writing giveaway_gamer.py


In [17]:
%%writefile most_frequent_counteracting.py
def most_frequent_counteracting(observation, configuration):
# Agent learns to counteract the most frequent move of the opponent
    global arr_lastOpponentAction
    if observation.step == 0:
        arr_lastOpponentAction = []
        return
    action = observation.lastOpponentAction
    arr_lastOpponentAction.append(action)
    i = arr_lastOpponentAction.count(0)
    j = arr_lastOpponentAction.count(1)
    k = arr_lastOpponentAction.count(2)
    n = max(i,j,k)
    if n == 0:
        return 1
    elif n == 1:
        return 2
    elif n == 2:
        return 0

Writing most_frequent_counteracting.py


In [18]:
%%writefile counteracting_giveaway.py
# Agent randomly switches its stratefy from counteracting to giving way to the most frequent action of the opponent  
def counteracting_giveaway(observation, configuration):
    import random
    global arr_lastOpponentAction
    if observation.step == 0:
        arr_lastOpponentAction = []
        return
    action = observation.lastOpponentAction
    arr_lastOpponentAction.append(action)
    i = arr_lastOpponentAction.count(0)
    j = arr_lastOpponentAction.count(1)
    k = arr_lastOpponentAction.count(2)
    n = max(i,j,k)
    if n == 0:
        return random.randint(1,2)
    elif n == 1:
        return random.randint(0,2)
    elif n == 2:
        return random.randint(1,2)

Writing counteracting_giveaway.py


In [19]:
%%writefile less_frequent_counteracting.py
def less_frequent_counteracting(observation, configuration):
# Agent learns to counteract the less frequent move of the opponent
    global arr_lastOpponentAction2
    if observation.step == 0:
        arr_lastOpponentAction2 = []
        return
    action = observation.lastOpponentAction
    arr_lastOpponentAction2.append(action)
    i = arr_lastOpponentAction2.count(0)
    j = arr_lastOpponentAction2.count(1)
    k = arr_lastOpponentAction2.count(2)
    n = min(i,j,k)
    if n == 0:
        return 1
    elif n == 1:
        return 2
    elif n == 2:
        return 0

Writing less_frequent_counteracting.py


In [20]:
%%writefile less_frequent_giveaway.py
def less_frequent_giveaway(observation, configuration):
# Agent learns to give away the less frequent move of the opponent
    global arr_lastOpponentAction3
    if observation.step == 0:
        arr_lastOpponentAction3 = []
        return
    action = observation.lastOpponentAction
    arr_lastOpponentAction3.append(action)
    i = arr_lastOpponentAction3.count(0)
    j = arr_lastOpponentAction3.count(1)
    k = arr_lastOpponentAction3.count(2)
    n = min(i,j,k)
    if n == 0:
        return 2
    elif n == 1:
        return 0
    elif n == 2:
        return 1

Writing less_frequent_giveaway.py


In [42]:
functions_dict = {0:'always_rock.py', 1: 'always_paper.py', 2: 'always_scissors.py', 3: 'random.py', 4: 'copy_opponent_old.py', 
                  5: 'statistical_old.py', 6: 'random_strategy_switch.py', 7: 'counteracting.py', 8: 'giveaway_gamer.py', 
                  9: 'most_frequent_counteracting.py', 10: 'counteracting_giveaway.py', 11: 'less_frequent_counteracting.py', 
                  12: 'less_frequent_giveaway.py'}
for i in range(len(functions_dict)):
    for j in range(len(functions_dict)):
        functions_lst = []
        functions_lst.append(functions_dict[i])
        functions_lst.append(functions_dict[j])
        evaluate("rps", functions_lst,configuration={"episodeSteps": 10})     

In [ ]:
functions_dict = {0:'always_rock.py', 1: 'always_paper.py', 2: 'always_scissors.py', 3: 'random.py', 4: 'copy_opponent_old.py', 
                  5: 'statistical_old.py', 6: 'random_strategy_switch.py', 7: 'counteracting.py', 8: 'giveaway_gamer.py', 
                  9: 'most_frequent_counteracting.py', 10: 'counteracting_giveaway.py', 11: 'less_frequent_counteracting.py', 
                  12: 'less_frequent_giveaway.py'}
for i in range(len(functions_dict)):
    for j in range(len(functions_dict)):
        functions_lst = []
        functions_lst.append(functions_dict[i])
        functions_lst.append(functions_dict[j])
        rewards = evaluate("rps", functions_lst, configuration={"episodeSteps": 100})
        print(f'{[functions_lst]} : {rewards}')